# Layer Visualizations
This notebook contains code to parse a saved TensorFlow model file and generate a pandas dataframe that contains information about all the layers.

In [1]:
import pandas as pd
import numpy as np
import re
from matplotlib import pyplot as plt
import plotly.express as px
from pathlib import Path

# enable plotly in VS Studio Code
import plotly.io as pio
#pio.renderers.default = "notebook_connected"
pio.renderers.default = "plotly_mimetype+notebook"

import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint

from workbench.utils.utils import create_filepaths, parse_model_name, append_dict_to_csv
from workbench.config.config import initialize
from workbench.tflite_benchmarking import get_profiling_stats_cpu, get_profiling_dataframes_cpu
from workbench.tflite_profiling import get_peak_memory_df, get_tensor_details_df

In [2]:
# Configure pandas to show all columns & rows
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [3]:
#model_name = "shufflenetv2tiny_0.1_96_c3_o3_f4l1024"
model_name = "mobilenetv1_0.1_96_c3_o3_l2"
#model_name = "efficientNetB0_0.1_96_c3_o3_keras"

In [4]:
model_dict = {}

In [5]:
parse_model_name(model_name)

('mobilenetv1', '0.1', '96', 'c3', 'o3', 'l2')

In [6]:
models_dir = initialize()

In [7]:
models_path, models_summary_path, models_image_path, models_layer_df_path, models_tf_path, models_tflite_path, models_tflite_opt_path = create_filepaths(model_name)

i:\tinyml\tiny_cnn\models


In [8]:
tflite_benchmark_cpu_path = models_dir.joinpath(model_name, f"{model_name}_benchmark.txt")

In [9]:
tflite_model_stats_cpu = get_profiling_stats_cpu(tflite_benchmark_cpu_path)
tflite_model_stats_cpu

{'model_size_MB': 0.07144,
 'init_us': 154788,
 'first_inference_us': 2826,
 'warmup_avg_us': 677.976,
 'inference_avg_us': 674.613,
 'initialization_ms': 154.788,
 'modify_graph_with_delegate_ms_first': 0.083,
 'modify_graph_with_delegate_ms_avg': 0.083,
 'modify_graph_with_delegate_ms_%': 64.341,
 'modify_graph_with_delegate_mem_KB': 0.0,
 'allocate_tensors_ms_first': 0.046,
 'allocate_tensors_ms_avg': 0.046,
 'allocate_tensors_ms_%': 35.659}

In [10]:
operator_run_order_df, operator_by_comp_time_df, summary_node_type_df = get_profiling_dataframes_cpu(tflite_benchmark_cpu_path, model_name)

In [11]:
operator_run_order_df

,node type,first,avg ms,%,cdf%,mem KB,times called,Name
1,QUANTIZE,0.104,0.105,15.945,15.945,0.0,1,[tfl.quantize]:0
2,CONV_2D,0.094,0.091,13.832,29.777,0.0,1,[mobilenetv1/re_lu/Relu;mobilenetv1/batch_norm...
3,DEPTHWISE_CONV_2D,0.108,0.115,17.609,47.386,0.0,1,[mobilenetv1/re_lu_1/Relu;mobilenetv1/batch_no...
4,CONV_2D,0.043,0.045,6.925,54.312,0.0,1,[mobilenetv1/re_lu_2/Relu;mobilenetv1/batch_no...
5,DEPTHWISE_CONV_2D,0.046,0.050,7.618,61.929,0.0,1,[mobilenetv1/re_lu_3/Relu;mobilenetv1/batch_no...
6,CONV_2D,0.015,0.017,2.559,64.488,0.0,1,[mobilenetv1/re_lu_4/Relu;mobilenetv1/batch_no...
7,DEPTHWISE_CONV_2D,0.046,0.050,7.589,72.077,0.0,1,[mobilenetv1/re_lu_5/Relu;mobilenetv1/batch_no...
8,CONV_2D,0.017,0.018,2.728,74.805,0.0,1,[mobilenetv1/re_lu_6/Relu;mobilenetv1/batch_no...
9,DEPTHWISE_CONV_2D,0.019,0.020,3.001,77.807,0.0,1,[mobilenetv1/re_lu_7/Relu;mobilenetv1/batch_no...
10,CONV_2D,0.007,0.008,1.189,78.996,0.0,1,[mobilenetv1/re_lu_8/Relu;mobilenetv1/batch_no...


In [12]:
operator_by_comp_time_df

,node type,first,avg ms,%,cdf%,mem KB,times called,Name
1,DEPTHWISE_CONV_2D,0.108,0.115,17.609,17.609,0.0,1,[mobilenetv1/re_lu_1/Relu;mobilenetv1/batch_no...
2,QUANTIZE,0.104,0.105,15.945,33.554,0.0,1,[tfl.quantize]:0
3,CONV_2D,0.094,0.091,13.832,47.386,0.0,1,[mobilenetv1/re_lu/Relu;mobilenetv1/batch_norm...
4,DEPTHWISE_CONV_2D,0.046,0.050,7.618,55.004,0.0,1,[mobilenetv1/re_lu_3/Relu;mobilenetv1/batch_no...
5,DEPTHWISE_CONV_2D,0.046,0.050,7.589,62.593,0.0,1,[mobilenetv1/re_lu_5/Relu;mobilenetv1/batch_no...
6,CONV_2D,0.043,0.045,6.925,69.518,0.0,1,[mobilenetv1/re_lu_2/Relu;mobilenetv1/batch_no...
7,DEPTHWISE_CONV_2D,0.028,0.030,4.590,74.108,0.0,1,[mobilenetv1/re_lu_9/Relu;mobilenetv1/batch_no...
8,DEPTHWISE_CONV_2D,0.019,0.020,3.001,77.109,0.0,1,[mobilenetv1/re_lu_7/Relu;mobilenetv1/batch_no...
9,CONV_2D,0.017,0.018,2.728,79.838,0.0,1,[mobilenetv1/re_lu_6/Relu;mobilenetv1/batch_no...
10,CONV_2D,0.015,0.017,2.559,82.396,0.0,1,[mobilenetv1/re_lu_4/Relu;mobilenetv1/batch_no...


In [13]:
summary_node_type_df

,Node type,count,avg ms,avg %,cdf %,mem KB,times called
1,DEPTHWISE_CONV_2D,10,0.309,48.433,48.433,0.0,10
2,CONV_2D,12,0.221,34.639,83.072,0.0,12
3,QUANTIZE,2,0.104,16.301,99.373,0.0,2
4,MEAN,1,0.002,0.313,99.687,0.0,1
5,STRIDED_SLICE,1,0.001,0.157,99.843,0.0,1
6,PACK,1,0.001,0.157,100.000,0.0,1
7,SOFTMAX,1,0.000,0.000,100.000,0.0,1
8,SHAPE,1,0.000,0.000,100.000,0.0,1
9,RESHAPE,1,0.000,0.000,100.000,0.0,1


# Read model layers file

In [ ]:
filepath = models_dir.joinpath(f"{model_name}/{model_name}_layers.pkl")

#filepath = f"i:/tinyml/tiny_cnn/models/{model_name}/{model_name}_layers.pkl"


In [ ]:
try:
    df = pd.read_pickle(filepath)
except:
    print(f"Try reading the file as csv")
    df = pd.read_csv(filepath)

In [ ]:
df.columns

In [ ]:
df.head(10)

## Helper functions

In [ ]:
def split_tuples(x):
    x = str(x).strip("()")
    x = x.split(",")
    return x

In [ ]:
def return_batch_size(x):
    x = str(x).strip("()")
    split_strings = x.split(",")
    #print(split_strings)
    return split_strings[0].strip()

In [ ]:
def return_input_height(x):
    #image_size=(img_height, img_width),

    x = str(x).strip("()")
    split_strings = x.split(",")
    if len(split_strings) in [3,4]:
        return split_strings[1].strip()
    else:
        return "None"


In [ ]:
def return_input_width(x):
    #image_size=(img_height, img_width),

    x = str(x).strip("()")
    split_strings = x.split(",")
    if len(split_strings) in [4]:
        return split_strings[2].strip()
    else:
        return "None"


In [ ]:
def return_channels(x):
    x = str(x).strip("()")
    split_strings = x.split(",")
    #print(split_strings)

    return split_strings[-1].strip()

In [ ]:
def split_shape_columns(df, colum_name):
    df[colum_name] = df[colum_name].map(str)
    df[colum_name] = df[colum_name].str.strip("[]").replace('\)\,', ')*', regex=True)
    df_cols = df[colum_name].str.split("*", expand=True)
    prefix = colum_name.split("_")[0]
    df_cols.columns = [f"{prefix}_{x+1}" for x in df_cols.columns]
    return df_cols

In [ ]:
# df["input_shape"].unique()

In [ ]:
# split_df = df["input_shape"].map(str)
# split_df.unique

# Input shapes & activations

In [ ]:
df_input = split_shape_columns(df, "input_shape")
no_input_cols = len(df_input.columns)
for input in range(1,no_input_cols+1):
    df_input[f"b_i_{input}"]= df_input[f"input_{input}"].apply(return_batch_size)
    df_input[f"h_i_{input}"] = df_input[f"input_{input}"].apply(return_input_height)
    df_input[f"w_i_{input}"] = df_input[f"input_{input}"].apply(return_input_width)
    df_input[f"c_i_{input}"] = df_input[f"input_{input}"].apply(return_channels)


input_cols = df_input.columns.to_list()

#df_input["w_i_2"].replace('None', np.nan, inplace=True)
df_input["b_i_1"].replace('None', np.nan, inplace=True)
df_input["w_i_1"].replace('None', np.nan, inplace=True)
df_input["h_i_1"].replace('None', np.nan, inplace=True)
df_input["c_i_1"].replace('None', np.nan, inplace=True)

if "b_i_2" in input_cols:
    df_input["b_i_2"] = df_input["b_i_2"].str.replace("\(", "")
    df_input["b_i_2"].replace('None', np.nan, inplace=True)
    df_input["w_i_2"].replace('None', np.nan, inplace=True)
    df_input["h_i_2"].replace('None', np.nan, inplace=True)
    df_input["c_i_2"].replace('None', np.nan, inplace=True)
df_input.fillna(1, inplace=True)

# cast to int
df_input["w_i_1"] = df_input["w_i_1"].map(int)
df_input["h_i_1"] = df_input["h_i_1"].map(int)
df_input["b_i_1"] = df_input["b_i_1"].map(int)
df_input["c_i_1"] = df_input["c_i_1"].map(int)

if "b_i_2" in input_cols:
    df_input["b_i_2"] = df_input["b_i_2"].map(int)
    df_input["w_i_2"] = df_input["w_i_2"].map(int)
    df_input["h_i_2"] = df_input["h_i_2"].map(int)
    df_input["c_i_2"] = df_input["c_i_2"].map(int)

In [ ]:
# df_input = split_shape_columns(df, "input_shape")
# df_input

In [ ]:
# #df["input_shape"][0] =(None, None, None, None)
# df["input_shape"] = df["input_shape"].map(str)
# df["input_shape"] = df["input_shape"].str.strip("[]").replace('\)\,', ')*', regex=True)
# df_input = df["input_shape"].str.split("*", expand=True)
# df_input.columns =[f"input_{x+1}" for x in df_input.columns]
# no_input_cols = len(df_input.columns)
# print(no_input_cols)
# df_input


In [ ]:
#df_input["input_1_split"] = df_input["input_1"].apply(split_tuples)

In [ ]:
# no_input_cols = len(df_input.columns)

In [ ]:
# for input in range(1,no_input_cols+1):
#     df_input[f"b_i_{input}"]= df_input[f"input_{input}"].apply(return_batch_size)
#     df_input[f"h_i_{input}"] = df_input[f"input_{input}"].apply(return_input_height)
#     df_input[f"w_i_{input}"] = df_input[f"input_{input}"].apply(return_input_width)
#     df_input[f"c_i_{input}"] = df_input[f"input_{input}"].apply(return_channels)

# df_input

In [ ]:
# input_cols = df_input.columns.to_list()


In [ ]:
# #df_input["w_i_2"].replace('None', np.nan, inplace=True)
# df_input["b_i_1"].replace('None', np.nan, inplace=True)
# df_input["w_i_1"].replace('None', np.nan, inplace=True)
# df_input["h_i_1"].replace('None', np.nan, inplace=True)
# df_input["c_i_1"].replace('None', np.nan, inplace=True)

# if "b_i_2" in input_cols:
#     df_input["b_i_2"] = df_input["b_i_2"].str.replace("\(", "")
#     df_input["b_i_2"].replace('None', np.nan, inplace=True)
#     df_input["w_i_2"].replace('None', np.nan, inplace=True)
#     df_input["h_i_2"].replace('None', np.nan, inplace=True)
#     df_input["c_i_2"].replace('None', np.nan, inplace=True)
# df_input.fillna(1, inplace=True)


In [ ]:
# df_input

In [ ]:
#df_input.c_i_1.value_counts()

In [ ]:

# df_input["w_i_1"] = df_input["w_i_1"].map(int)
# df_input["h_i_1"] = df_input["h_i_1"].map(int)
# df_input["b_i_1"] = df_input["b_i_1"].map(int)
# df_input["c_i_1"] = df_input["c_i_1"].map(int)

# if "b_i_2" in input_cols:
#     df_input["b_i_2"] = df_input["b_i_2"].map(int)
#     df_input["w_i_2"] = df_input["w_i_2"].map(int)
#     df_input["h_i_2"] = df_input["h_i_2"].map(int)
#     df_input["c_i_2"] = df_input["c_i_2"].map(int)

In [ ]:

# df_input["b_i_1"] = df_input["b_i_1"].map(int)
# df_input["w_i_1"] = df_input["w_i_1"].map(int)
# df_input["h_i_1"] = df_input["h_i_1"].map(int)
# df_input["c_i_1"] = df_input["c_i_1"].map(int)

# df_input["b_i_2"] = df_input["b_i_2"].map(int)
# df_input["w_i_2"] = df_input["w_i_2"].map(int)
# df_input["h_i_2"] = df_input["h_i_2"].map(int)
# df_input["c_i_2"] = df_input["c_i_2"].map(int)

In [ ]:
#df_input

# Output shape and activations

In [ ]:
df_output =split_shape_columns(df, "output_shape") # TODO: This split does not work for EfficientNet
no_output_cols = len(df_output.columns)
# split output dimensions
for output in range(1,no_output_cols+1):
    df_output[f"b_o_{output}"]= df_output[f"output_{output}"].apply(return_batch_size)
    df_output[f"h_o_{output}"] = df_output[f"output_{output}"].apply(return_input_height)
    df_output[f"w_o_{output}"] = df_output[f"output_{output}"].apply(return_input_width)
    df_output[f"c_o_{output}"] = df_output[f"output_{output}"].apply(return_channels)

# fill NaN with 1 for mulitplication
df_output["b_o_1"].replace('None', np.nan, inplace=True)
df_output["w_o_1"].replace('None', np.nan, inplace=True)
df_output["h_o_1"].replace('None', np.nan, inplace=True)
df_output["c_o_1"].replace('None', np.nan, inplace=True)
df_output.fillna(1, inplace=True)

df_output["w_o_1"] = df_output["w_o_1"].map(int)
df_output["h_o_1"] = df_output["h_o_1"].map(int)
df_output["b_o_1"] = df_output["b_o_1"].map(int)
df_output["c_o_1"] = df_output["c_o_1"].map(int)

In [ ]:
# df_output = split_shape_columns(df, "output_shape")
# df_output

In [ ]:
# no_output_cols = len(df_output.columns)
# no_output_cols

In [ ]:
# for output in range(1,no_output_cols+1):
#     df_output[f"b_o_{output}"]= df_output[f"output_{output}"].apply(return_batch_size)
#     df_output[f"h_o_{output}"] = df_output[f"output_{output}"].apply(return_input_height)
#     df_output[f"w_o_{output}"] = df_output[f"output_{output}"].apply(return_input_width)
#     df_output[f"c_o_{output}"] = df_output[f"output_{output}"].apply(return_channels)

# df_output

In [ ]:
# df_output["b_o_1"].replace('None', np.nan, inplace=True)
# df_output["w_o_1"].replace('None', np.nan, inplace=True)
# df_output["h_o_1"].replace('None', np.nan, inplace=True)
# df_output["c_o_1"].replace('None', np.nan, inplace=True)
# df_output.fillna(1, inplace=True)

In [ ]:
# df_output["w_o_1"] = df_output["w_o_1"].map(int)
# df_output["h_o_1"] = df_output["h_o_1"].map(int)
# df_output["b_o_1"] = df_output["b_o_1"].map(int)
# df_output["c_o_1"] = df_output["c_o_1"].map(int)

# Feature Engineering

In [ ]:
def get_unique_values_without_nan(df,col_name):
    """Retrieve list of unique column values, excluding nan

    Args:
        df (pd.DataFrame): pandas dataframe that contains the required column
        col_name (string): name of a column in the pandas dataframe

    Returns:
        list: list of extracted unique values of the specified column
    """
    try:
        value_list = df[col_name].unique().tolist()
        # exclude nan
        value_list = [item for item in value_list if str(item) != 'nan']
        return value_list
    except:
        print(f"WARNING: Colum name {col_name} is not in this dataframe!")

        return []

In [ ]:
df = df.join(df_input)

In [ ]:
df = df.join(df_output)

In [ ]:
name_code ="i_2"

In [ ]:
def calculate_activations(x, name_code):
    h_i = int(x[f"h_{name_code}"])
    w_i = int(x[f"w_{name_code}"])
    c_i = int(x[f"c_{name_code}"])
    out = h_i * w_i * c_i
    if out >1:
        return out
    else:
        return 0

In [ ]:
df["#i_1_activations"] = df.apply(calculate_activations, name_code= "i_1", axis=1)#df_input["h_i_1"] * df_input["w_i_1"] * df_input["c_i_1"]
df["#o_1_activations"] = df.apply(calculate_activations, name_code= "o_1", axis=1)#  df_output["h_o_1"] * df_output["w_o_1"] * df_output["c_o_1"]

In [ ]:
if "b_i_2" in input_cols:
    df["#i_2_activations"] = df.apply(calculate_activations, name_code="i_2", axis=1)

In [ ]:
df["#i_2_activations"].value_counts()

In [ ]:
if "#i_2_activations" in df.columns:
    df["peak_activations"] = df["#i_1_activations"] + df["#i_2_activations"] + df["#o_1_activations"]
else:
    df["peak_activations"] = df["#i_1_activations"]  + df["#o_1_activations"]    

In [ ]:
unique_values = get_unique_values_without_nan(df, "input_node_2")
unique_values

In [ ]:
# fill activations from start node to end node

for item in unique_values:
    first_idx = df.loc[df["name"] == item].index[0]
    last_idx = df.loc[df["input_node_2"] == item].index[0]

    activations = df["#i_2_activations"][last_idx]

    # fill the corresponding rows with the extracted information
    df.loc[first_idx: last_idx, "#i_2_activations"] = activations
print(df[first_idx: last_idx+1])

In [ ]:
df.columns

In [ ]:
df["total_params"] = df["params"].cumsum()
df["total_MACs"] = df["MACS"].cumsum()
df["total_OPS"] = df["OPS"].cumsum()

In [ ]:
# df["total_params"].max()
# df["peak_activations"].argmax()

In [ ]:
model_dict["model_name"] = model_name
model_dict["total_params"] = df["total_params"].max()
model_dict["total_MACs"] = df["total_MACs"].max()
model_dict["total OPS"] = df["total_OPS"].max()
model_dict["peak_activations"] = df["peak_activations"].max()
model_dict["index_peak_activations"] = df["peak_activations"].argmax()

In [ ]:
model_dict

In [ ]:
csv_file_name = "model_metrics.csv"
csv_path = Path.cwd().joinpath(csv_file_name)
append_dict_to_csv(csv_path, model_dict)

In [ ]:
# file path for the cleaned dataframe
df_filepath = models_dir.joinpath(f"{model_name}/{model_name}.pkl")

In [ ]:
df.to_pickle(df_filepath)

In [ ]:
df.columns

In [ ]:
df.head(12)

In [ ]:
df["input_shape"].unique()

## Working with Activations

In [ ]:
# unique_values = get_unique_values_without_nan(df, "input_node_0")
# unique_values

In [ ]:

# # fill activations from start node to end node

# for item in unique_values:
#     first_idx = df.loc[df["name"] == item].index[0]
#     last_idx = df.loc[df["input_node_0"] == item].index[0]

#     activations = df["#i_2_activations"][last_idx]

#     # fill the corresponding rows with the extracted information
#     df.loc[first_idx: last_idx, "#i_2_activations"] = activations
#     #print(df[first_idx: last_idx+1])

In [ ]:
# df.columns

In [ ]:
# activation_cols = ["index", "name", "input_shape", "input_node_1", "#i_1_activations" , "input_node_0", "#i_2_activations", "input_shape", "input_node_type_1", "input_node_type_0"]
# df_activations = df[activation_cols].copy()
# df_activations.head(20)

In [ ]:
# # fill activations from start node to end node

# for item in second_inputs:
#     first_idx = df_activations.loc[df_activations["name"] == item].index[0]
#     last_idx = df_activations.loc[df_activations["input_node_0"] == item].index[0]

#     activations = df_activations["#i_2_activations"][last_idx]
#     df_activations.loc[first_idx: last_idx, "#i_2_activations"] = activations
#     #print(df_activations[first_idx: last_idx+1])
    

    

In [ ]:
#input_name = "block_4_project_BN"

In [ ]:
# first_idx = df.loc[df["name"] == input_name].index[0]
# last_idx = df.loc[df["input_node_2"] == input_name].index[0]
# last_idx

In [ ]:
df[first_idx: last_idx+1]

In [ ]:
# df_activations.loc[last_idx]

In [ ]:
# activations = df_activations["#i_2_activations"][last_idx]
# activations

In [ ]:
# df_activations.loc[first_idx: last_idx, "#i_2_activations"] = activations
# df_activations.head(30)

## Save cleaned dataframe

In [ ]:
clean_df_filepath = models_dir.joinpath(f"{model_name}/{model_name}_layers_clean.pkl")

#filepath = f"i:/tinyml/tiny_cnn/models/{model_name}/{model_name}_layers.pkl"
df.to_pickle(clean_df_filepath)

# Load tflite model data

In [14]:
#models_peak_memory_path = models_dir.joinpath(model_name, f"{model_name}_peak_memory")
model_peak_memory_schedule = models_dir.joinpath(model_name, f"{model_name}_peak_memory", "execution_schedule_info.csv")
peak_memory_tensor_details = models_dir.joinpath(model_name, f"{model_name}_peak_memory", "tensor_details.csv")

In [15]:
peak_memory_df = get_peak_memory_df(model_peak_memory_schedule)
peak_memory_df

Reading in i:\tinyml\tiny_cnn\models\mobilenetv1_0.1_96_c3_o3_l2\mobilenetv1_0.1_96_c3_o3_l2_peak_memory\execution_schedule_info.csv
Cleaning up the dataframe.


,name,tensor_IDs,RAM_b,operator
0,tfl.quantize,"(0, 49)",55296,tfl.quantize
1,conv2d,"(49, 50)",34560,mobilenetv1/re_lu/Relu;mobilenetv1/batch_norma...
2,conv2d_11,"(50, 51)",13824,mobilenetv1/re_lu_1/Relu;mobilenetv1/batch_nor...
3,conv2d_1,"(51, 52)",20736,mobilenetv1/re_lu_2/Relu;mobilenetv1/batch_nor...
4,depthwise_conv2d_1,"(52, 53)",17280,mobilenetv1/re_lu_3/Relu;mobilenetv1/batch_nor...
5,conv2d_2,"(53, 54)",10368,mobilenetv1/re_lu_4/Relu;mobilenetv1/batch_nor...
6,depthwise_conv2d_3,"(54, 55)",13824,mobilenetv1/re_lu_5/Relu;mobilenetv1/batch_nor...
7,conv2d_3,"(55, 56)",13824,mobilenetv1/re_lu_6/Relu;mobilenetv1/batch_nor...
8,depthwise_conv2d_3,"(56, 57)",8640,mobilenetv1/re_lu_7/Relu;mobilenetv1/batch_nor...
9,conv2d_4,"(57, 58)",5328,mobilenetv1/re_lu_8/Relu;mobilenetv1/batch_nor...


In [16]:
# Get dataframes from benchmarking tflite model 
models_peak_memory_df_path = models_dir.joinpath(model_name, "peak_memory_df")
models_peak_memory_df_path

tensor_info_df_path = models_dir.joinpath(model_name, "tensor_info_df")
tensor_info_df_path

WindowsPath('i:/tinyml/tiny_cnn/models/mobilenetv1_0.1_96_c3_o3_l2/tensor_info_df')

In [17]:
tensor_df = get_tensor_details_df(peak_memory_tensor_details)
tensor_df

Reading in i:\tinyml\tiny_cnn\models\mobilenetv1_0.1_96_c3_o3_l2\mobilenetv1_0.1_96_c3_o3_l2_peak_memory\tensor_details.csv
Cleaning up the dataframe.


,id,name,shape,size_b,name_long
0,0,serving_default_input_1:0,"(1, 96, 96, 3)",27648,serving_default_input_1:0
1,49,tfl.quantize,"(1, 96, 96, 3)",27648,tfl.quantize
2,50,mobilenetv1/conv2d/Conv2D1,"(1, 48, 48, 3)",6912,mobilenetv1/re_lu/Relu;mobilenetv1/batch_norma...
3,51,mobilenetv1/conv2d_11/Conv2D1,"(1, 48, 48, 3)",6912,mobilenetv1/re_lu_1/Relu;mobilenetv1/batch_nor...
4,52,mobilenetv1/conv2d_1/Conv2D1,"(1, 48, 48, 6)",13824,mobilenetv1/re_lu_2/Relu;mobilenetv1/batch_nor...
5,53,mobilenetv1/depthwise_conv2d_1/BiasAdd1,"(1, 24, 24, 6)",3456,mobilenetv1/re_lu_3/Relu;mobilenetv1/batch_nor...
6,54,mobilenetv1/conv2d_2/Conv2D1,"(1, 24, 24, 12)",6912,mobilenetv1/re_lu_4/Relu;mobilenetv1/batch_nor...
7,55,mobilenetv1/depthwise_conv2d_3/depthwise1,"(1, 24, 24, 12)",6912,mobilenetv1/re_lu_5/Relu;mobilenetv1/batch_nor...
8,56,mobilenetv1/conv2d_3/Conv2D1,"(1, 24, 24, 12)",6912,mobilenetv1/re_lu_6/Relu;mobilenetv1/batch_nor...
9,57,mobilenetv1/depthwise_conv2d_3/BiasAdd1,"(1, 12, 12, 12)",1728,mobilenetv1/re_lu_7/Relu;mobilenetv1/batch_nor...


In [ ]:
# if tensor_info_df_path.is_file:
#     tensor_info_df = pd.read_pickle(tensor_info_df_path)
#     tensor_info_df
# else:
#     print(f"This file does not exist: {tensor_info_df_path}")

In [ ]:
# peak_memory_df = pd.read_pickle(models_peak_memory_df_path)
# peak_memory_df

# Visualizations

In [ ]:
# fig = px.bar(df, x="name", y= ["#i_1_activations","#i_2_activations", "#o_1_activations"], text_auto=".2s",
# width=1400, height=500,
#     title=f"Peak activations per layer - {model_name}")
# fig.add_hline(y=256000, line_width=3, line_dash="dash", line_color="orange", annotation_text="256 kB MCU constraint", 
#               annotation_position="bottom right")
# fig.update_layout(yaxis_range=[0,300000])
# #fig.update_layout(showlegend=True)
# fig.show()

In [32]:
peak_mem_fig = px.bar(peak_memory_df, x="operator", y= ["RAM_b"], text_auto=".2s",
width=1400, height=600,
    title=f"Peak activations per layer - tflite {model_name}")
# tflite_fig.add_hline(y=256, line_width=3, line_dash="dash", line_color="orange", annotation_text="256 kB MCU constraint", 
#               annotation_position="bottom right")
peak_memory = peak_memory_df["RAM_b"].max()
peak_mem_fig.add_hline(y=peak_memory, line_width=3, line_dash="dash", line_color="red", annotation_text=f"{peak_memory} B peak activation", 
               annotation_position="top")
# tflite_fig.update_layout(yaxis_range=[0,300])
peak_mem_fig.update_traces(width=1)
peak_mem_fig.update_layout(showlegend=True)
peak_mem_fig.show()

In [18]:
# tflite_fig = px.bar(peak_memory_df, x="operator", y= ["peak_memory_kb"], text_auto=".2s",
# width=1400, height=600,
#     title=f"Peak activations per layer - tflite {model_name}")
# tflite_fig.add_hline(y=256, line_width=3, line_dash="dash", line_color="orange", annotation_text="256 kB MCU constraint", 
#               annotation_position="bottom right")
# peak_memory = peak_memory_df["peak_memory_kb"].max()
# tflite_fig.add_hline(y=peak_memory, line_width=3, line_dash="dash", line_color="red", annotation_text=f"{peak_memory} kB peak activation", 
#               annotation_position="top")
# tflite_fig.update_layout(yaxis_range=[0,300])
# #tflite_fig.update_layout(showlegend=True)
# tflite_fig.show()

ValueError: All arguments should have the same length. The length of argument `y` is 1, whereas the length of  previously-processed arguments ['operator'] is 30

In [ ]:
fig6 = px.bar(df, x="name", y= "MACS",
    title=f"MACs per layer - {model_name}")
#fig.add_hline(y=256000, line_width=3, line_dash="dash", line_color="orange", annotation_text="256 kB MCU constraint", 
#              annotation_position="bottom right")
#fig.update_layout(showlegend=True)
fig6.show()

In [ ]:
fig2 = px.bar(df, x="name", y= "h_i_1", text_auto=".2s",
    title=f"Input height per layer - {model_name}")
#fig2.add_hline(y=256000, line_width=3, line_dash="dash", line_color="orange")
#fig2.update_yaxes(autorange="reversed")
#fig2.update_yaxes(rangemode="tozero")
fig2.show()

In [ ]:
fig3 = px.bar(df, x="name", y= "c_i_1", text_auto=".2s",
    title=f"Channels per layer - {model_name}")
#fig2.add_hline(y=256000, line_width=3, line_dash="dash", line_color="orange")
#fig3.update_yaxes(autorange="reversed")
#fig3.update_yaxes(rangemode="tozero")
# fig3.update_yaxes(
#     range=(0, 1280),
#     constrain='domain'
# )
fig3.show()

In [ ]:
fig4 = px.line(df, x="name", y= "total_params",
    title=f"Cumulative parameters per layer - {model_name}")
#fig.add_hline(y=256000, line_width=3, line_dash="dash", line_color="orange", annotation_text="256 kB MCU constraint", 
#              annotation_position="bottom right")
#fig.update_layout(showlegend=True)
fig4.show()

In [ ]:
fig5 = px.line(df, x="name", y= "total_MACs",
    title=f"Cumulative MACs per layer - {model_name}")
#fig.add_hline(y=256000, line_width=3, line_dash="dash", line_color="orange", annotation_text="256 kB MCU constraint", 
#              annotation_position="bottom right")
#fig.update_layout(showlegend=True)
fig5.show()

In [ ]:
# # Generate run ids
# id = wandb.util.generate_id()

# PROJECT = model_name.split("_")[0]

# run = wandb.init(
#         # Set the project where this run will be logged
#         project=PROJECT, 
#         name = model_name,
#         id = id, 
#         resume="allow",
#         sync_tensorboard=True
#         )
# # Specify the configuration variables
# config = wandb.config

# #config.batch_size = BATCH_SIZE
# #config.dropout =DROPOUT
# #config.learn_rate = LR
# #config.momentum = MOMENTUM
# #config.decay = 1e-6
# #config.epochs = EPOCHS
# #config.classes = classes
# config.id = id
# config.architecture = model_name

# # Create a table
# table = wandb.Table(columns = ["plotly_figure"])

# # Create path for Plotly figure
# path_to_plotly_html = "./plotly_figure.html"

# # Write Plotly figure to HTML
# fig.write_html(path_to_plotly_html, auto_play = False) # Setting auto_play to False prevents animated Plotly charts from playing in the table automatically

# # Add Plotly figure as HTML file into Table
# table.add_data(wandb.Html(path_to_plotly_html))

# # Log Table
# run.log({"Chart_table": table})

# #wandb.log({"Peak activations chart": fig})

# wandb.finish()